In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_train.csv')
test_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.1, random_state=1)
test_data = test_data.sample(frac=0.1, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# VGG modelini oluştur
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg = models.Sequential()
model_vgg.add(base_model)
model_vgg.add(layers.Flatten())
model_vgg.add(layers.Dense(256, activation='relu'))
model_vgg.add(layers.Dropout(0.5))
model_vgg.add(layers.Dense(10, activation='softmax'))
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_vgg.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_vgg = model_vgg.predict(X_test_resized_rgb)
y_pred_vgg_classes = np.argmax(y_pred_vgg, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_vgg = accuracy_score(y_test_classes, y_pred_vgg_classes)
precision_vgg = precision_score(y_test_classes, y_pred_vgg_classes, average='weighted')
recall_vgg = recall_score(y_test_classes, y_pred_vgg_classes, average='weighted')
f1_vgg = f1_score(y_test_classes, y_pred_vgg_classes, average='weighted')

print(f"Accuracy: {accuracy_vgg}")
print(f"Precision: {precision_vgg}")
print(f"Recall: {recall_vgg}")
print(f"F1 Score: {f1_vgg}")


2023-12-23 14:23:54.833883: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 14:23:55.125201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 14:23:55.125274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 14:23:55.187284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-23 14:23:55.314053: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 14:23:55.315153: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/5


2023-12-23 14:25:27.307685: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2023-12-23 14:25:27.521200: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.


75/75 [==============================] - 2279s 30s/step - loss: 2.3559 - accuracy: 0.1158 - val_loss: 2.3021 - val_accuracy: 0.1175
Epoch 2/5
75/75 [==============================] - 2110s 28s/step - loss: 2.3012 - accuracy: 0.1156 - val_loss: 2.3021 - val_accuracy: 0.1175
Epoch 3/5
75/75 [==============================] - 2113s 28s/step - loss: 2.3003 - accuracy: 0.1156 - val_loss: 2.3025 - val_accuracy: 0.1175
Epoch 4/5
75/75 [==============================] - 2112s 28s/step - loss: 2.2998 - accuracy: 0.1156 - val_loss: 2.3027 - val_accuracy: 0.1175
Epoch 5/5
32/32 [==============================] - 125s 4s/step
Accuracy: 0.107
Precision: 0.011448999999999999
Recall: 0.107
F1 Score: 0.020684733514001807


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
